In [1]:
!nvidia-smi

Thu Sep 26 10:47:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:87:00.0 Off |                    0 |
| N/A   32C    P0              79W / 400W |      3MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [8]:
import torch

print("Cuda available: ", torch.cuda.is_available())
print("Number of GPUs: ", torch.cuda.device_count())

Cuda available:  True
Number of GPUs:  2


In [1]:
import gc
import json
import pandas as pd
# import numpy as np

from tqdm import tqdm
from random import sample 
from random import randint
from datasets import load_dataset
from transformers import Trainer
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling

In [2]:
dir_root = "Data/"

## Prepair Train

In [3]:
def generate_prompt(data_point):
    # taken from https://github.com/tloen/alpaca-lora
    if data_point["instruction"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
        
### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"]}"""

In [4]:
def generate_prompt_context(data_point):
    return f"""[INST]Below is an instruction that describes a task! Write a response that appropriately completes the request. To answer the instruction, use the context if available!
### Instruction: 
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Context: 
{data_point["context"]}
[/INST]

### Response:
{data_point["output"]}
"""

In [5]:
def generate_prompt_llama2(data_point):
    return f"""[INST]Below is an instruction that describes a task! Write a response that appropriately completes the request. To answer the instruction, use the context if available!
### Instruction: 
{data_point["instruction"]}

### Input:
{data_point["input"]}
[/INST]

### Response:
{data_point["output"]}
"""

In [6]:
def train(model, version, data_files, gp):
    try:
        output_dir = f"Models/{version}/"
        
        tokenizer = AutoTokenizer.from_pretrained(model)
        tokenizer.pad_token_id = tokenizer.eos_token_id
        print("Loaded: TOKENIZER")
    
        data = load_dataset("json", data_files=data_files)
        # data = data.map(lambda data_point: {"prompt": tokenizer(gp(data_point))})    
        data = data.shuffle().map(
            lambda data_point: tokenizer(
                gp(data_point),
                truncation=True,
                max_length=256,
                padding="max_length",
            )
        )
        print("Loaded: DATA")
        
        model = AutoModelForCausalLM.from_pretrained(model, device_map="auto")
        print("Loaded: MODEL")

        # Enable gradient checkpointing to reduce memory usage
        model.gradient_checkpointing_enable()
        
        train_arguments = TrainingArguments(
            per_device_train_batch_size=4,
            gradient_accumulation_steps=16,
            warmup_steps=2,
            num_train_epochs=5, # paper uses 3
            learning_rate=2e-4,
            logging_steps=1,
            output_dir=output_dir,
            eval_strategy="epoch",
            save_strategy="epoch",
            save_total_limit=10,
            fp16=True)
    
        trainer = Trainer(
            model=model,
            train_dataset=data["train"],
            eval_dataset=data["test"],
            args=train_arguments,
            data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        )
        print("Loaded: TRAINER")
    
        
        model.config.use_cache = False
        trainer.train(resume_from_checkpoint=False)
    
        model.save_pretrained(f"{output_dir}endpoint")
        print("saved: MODEL")
        
        log_history = pd.DataFrame(trainer.state.log_history)
        log_history.to_csv(f"{dir_root}log_train_history_{version}.csv")
        print("saved: HISTORY")
    finally:
        model = None
        trainer = None
        train_arguments = None
        tokenizer = None
        data = None
        gc.collect()
        torch.cuda.empty_cache()

In [25]:
from IPython import get_ipython
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

## GPJ-B

In [9]:
train(model = "EleutherAI/gpt-j-6b",
      version = "gptj-6b-v20240926",
      data_files = {"train" : f"{dir_root}qaa_train.json", "test" : f"{dir_root}qaa_val.json"},
      gp = generate_prompt)

/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded: TOKENIZER


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10247 [00:00<?, ? examples/s]

...

Loaded: MODEL
Loaded: TRAINER


/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/envs/domaine/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/envs/domaine/lib/python3.10/site-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/util

Epoch,Training Loss,Validation Loss
0,1.156200,1.026613
1,0.732400,0.948013
2,0.273100,1.142371
3,0.108100,1.300829
4,0.061500,1.385511


/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defi

saved: MODEL
saved: HISTORY


## GPJ-B with cotext

In [10]:
train(model = "EleutherAI/gpt-j-6B",
      version = "gptj-6b-context-v20231216",
      data_files = {"train" : f"{dir_root}qaa_train_context.json", "test" : f"{dir_root}qaa_val_context.json"},
      gp = generate_prompt_context)

tokenizer_config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

...

Loaded: MODEL
Loaded: TRAINER


/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/envs/domaine/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,0.429500,0.759829
1,0.267900,0.705200
2,0.101100,0.819730
4,0.051800,0.940310


/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defi

saved: MODEL
saved: HISTORY


## opt-6.7b

In [11]:
train(model = "facebook/opt-6.7b",
      version = "opt-6.7b-v20240926",
      data_files = {"train" : f"{dir_root}qaa_train.json", "test" : f"{dir_root}qaa_val.json"},
      gp = generate_prompt)

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

...

Loaded: MODEL
Loaded: TRAINER


/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/envs/domaine/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,1.108200,1.180150
1,0.873100,1.002593
2,0.455000,1.024832
3,0.172400,1.096023
4,0.078700,1.191012


/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defi

saved: MODEL
saved: HISTORY


## opt-6.7b with context

In [ ]:
train(model = "facebook/opt-6.7b",
      version = "opt-6.7b-context-v20240927",
      data_files = {"train" : f"{dir_root}qaa_train_context.json", "test" : f"{dir_root}qaa_val_context.json"},
      gp = generate_prompt_context)

/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded: TOKENIZER


Map:   0%|          | 0/20494 [00:00<?, ? examples/s]

Map:   0%|          | 0/4536 [00:00<?, ? examples/s]

Loaded: DATA


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded: MODEL
Loaded: TRAINER


/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/envs/domaine/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,0.601500,0.850359
1,0.363800,0.713798
2,0.119400,0.745095
4,0.073100,0.784782


/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defi

## Llama-7b

In [13]:
train(model = "huggyllama/llama-7b",
      version = "llama-7b-v20240927",
      data_files = {"train" : f"{dir_root}qaa_train.json", "test" : f"{dir_root}qaa_val.json"},
      gp = generate_prompt)

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Loaded: MODEL
Loaded: TRAINER


/opt/conda/envs/domaine/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/envs/domaine/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,1.000300,1.032789


/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [14]:
train(model = "huggyllama/llama-7b",
      version = "llama-7b-context-v20240927",
      data_files = {"train" : f"{dir_root}qaa_train_context.json", "test" : f"{dir_root}qaa_val_context.json"},
      gp = generate_prompt_context)

Loaded: TOKENIZER


Map:   0%|          | 0/20494 [00:00<?, ? examples/s]

Map:   0%|          | 0/4536 [00:00<?, ? examples/s]

Loaded: DATA


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Loaded: TRAINER


/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,0.335300,0.589571
1,0.216500,0.551165
2,0.089400,0.632585
4,0.050500,0.759167


/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/e

saved: MODEL
saved: HISTORY


## Llama-2-7b-hf

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
train(model = "meta-llama/Llama-2-7b-hf",
      version = "llama-2-7b-hf-v20241001",
      data_files = {"train" : f"{dir_root}qaa_train.json", "test" : f"{dir_root}qaa_val.json"},
      gp = generate_prompt_llama2)

Loaded: TOKENIZER


Map:   0%|          | 0/10247 [00:00<?, ? examples/s]

Map:   0%|          | 0/2268 [00:00<?, ? examples/s]

Loaded: DATA


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

/opt/conda/envs/domaine/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/envs/domaine/lib/python3.10/site-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs)

Epoch,Training Loss,Validation Loss
0,0.834900,0.820101
1,0.497600,0.782055
2,0.203700,0.930519
3,0.081700,1.061301
4,0.051600,1.182239


/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defi

saved: MODEL
saved: HISTORY


NameError: name 'torch' is not defined

## Llama-2-7b-hf with context

In [22]:
train(model = "meta-llama/Llama-2-7b-hf",
      version = "llama-2-7b-hf-context-v20240927",
      data_files = {"train" : f"{dir_root}qaa_train_context.json", "test" : f"{dir_root}qaa_val_context.json"},
      gp = generate_prompt_context)

Loaded: TOKENIZER


Map:   0%|          | 0/20494 [00:00<?, ? examples/s]

Map:   0%|          | 0/4536 [00:00<?, ? examples/s]

Loaded: DATA


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded: MODEL
Loaded: TRAINER


/opt/conda/envs/domaine/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,0.348300,0.608959
1,0.223700,0.563524
2,0.091200,0.640563
4,0.049800,0.776762


/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/envs/domaine/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defi

saved: MODEL
saved: HISTORY
